In [1]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

In [2]:
results_dir = "/home/ec2-user/SageMaker/NAB/results-batadal/"

In [3]:
all_models = ["contextOSE"]
all_datasets = ["train_no_anomaly", "train_with_anomaly", "test_with_anomaly", "combined"]
all_thresholds = [0.05, 0.1, 0.25, 0.4, 0.5]

In [4]:
def _get_tp_tn_fp_fn(nab_output_df, threshold):
    predictions = nab_output_df["anomaly_score"].values >= threshold
    
    cnf_mat = confusion_matrix(nab_output_df["label"].values, predictions)
    
    FP = cnf_mat.sum(axis=0) - np.diag(cnf_mat)  
    FN = cnf_mat.sum(axis=1) - np.diag(cnf_mat)
    TP = np.diag(cnf_mat)
    TN = cnf_mat.sum() - (FP + FN + TP)
    
    return TP, TN, FP, FN

In [5]:
def get_tp_tn_fp_fn(predictions, labels):    
    cnf_mat = confusion_matrix(labels, predictions)
    
    FP = cnf_mat.sum(axis=0) - np.diag(cnf_mat)  
    FN = cnf_mat.sum(axis=1) - np.diag(cnf_mat)
    TP = np.diag(cnf_mat)
    TN = cnf_mat.sum() - (FP + FN + TP)
    
    return TP, TN, FP, FN

In [6]:
from scipy import stats

class CustomVotingClf:
    def __init__(self, thresh):
        self.thresh = thresh
        self.predictions = []
        self.labels = []
        
    def append(self, anomaly_scores, labels):
        predictions = anomaly_scores >= self.thresh
        self.predictions.append(predictions)
        
        if len(self.labels) > 0:
            assert np.array_equal(self.labels[-1], labels)
        
        self.labels.append(labels)
    
    def _voting(self, list_of_lists):
        npy = np.array(list_of_lists)
        mode = stats.mode(npy)[0]
        
        return mode
    
    def predict(self):
#         predictions_npy = np.array(self.predictions)
#         voting_pred = stats.mode(predictions_npy)[0]
        voting_pred = self._voting(self.predictions).squeeze()
        
        return voting_pred
    
    def evaluate(self):
        assert len(self.predictions) == len(self.labels)
        
        predictions = self.predict()
        #labels should be same across sensors as anomaly are at system level
        labels = self.labels[0]
        
        return get_tp_tn_fp_fn(predictions, labels)

In [7]:
parsed_results = []

for model in all_models:
    for data in all_datasets:
        classifiers_dict = {}
        for thresh in all_thresholds:
            classifiers_dict[thresh] = CustomVotingClf(thresh)
        
        results_csvs = glob.glob(os.path.join(results_dir, model, data, "*csv"))
        
        for csv_path in results_csvs:
            nab_result_df = pd.read_csv(csv_path)
            
            for thresh in all_thresholds:
                classifiers_dict[thresh].append(
                    nab_result_df["anomaly_score"].values, 
                    nab_result_df["label"].values
                    
                )
                
        
        for thresh in all_thresholds:
            tp, tn, fp, fn = classifiers_dict[thresh].evaluate()

            parsed_results.append(
                {
                    "model": model,
                    "dataset": data,
                    "threshold": thresh,
                    "tp": tp,
                    "tn": tn,
                    "fp": fp,
                    "fn": fn,
                    "tpr": np.round(tp/(tp + fn), 3),
                    "tnr": np.round(tn/(tn + fp), 3),
                    "fpr": np.round(fp/(fp + tn), 3),
                    "fnr": np.round(fn/(tp + fn), 3),
                    "f1": np.round(tp/(tp + 0.5*(fp + fn)), 3)
                    
                    
                }
            )
        
        
        

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:34: RuntimeWarning: invalid value encountered in true_divide
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:35: RuntimeWarning: invalid value encountered in true_divide
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:36: RuntimeWarning: invalid value encountered in true_divide
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:37: RuntimeWarning: invalid value encountered in true_divide
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:34: RuntimeWarning: invalid value encountered in true_divide
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:35: RuntimeWarning: invalid value encountered in true_divide
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:36: RuntimeWarning

In [8]:
pd.DataFrame(parsed_results)

,model,dataset,threshold,tp,tn,fp,fn,tpr,tnr,fpr,fnr,f1
0,contextOSE,train_no_anomaly,0.05,"[344, 0]","[0, 344]","[0, 8417]","[8417, 0]","[0.039, nan]","[nan, 0.039]","[nan, 0.961]","[0.961, nan]","[0.076, 0.0]"
1,contextOSE,train_no_anomaly,0.10,"[347, 0]","[0, 347]","[0, 8414]","[8414, 0]","[0.04, nan]","[nan, 0.04]","[nan, 0.96]","[0.96, nan]","[0.076, 0.0]"
2,contextOSE,train_no_anomaly,0.25,"[1413, 0]","[0, 1413]","[0, 7348]","[7348, 0]","[0.161, nan]","[nan, 0.161]","[nan, 0.839]","[0.839, nan]","[0.278, 0.0]"
3,contextOSE,train_no_anomaly,0.40,[8761],[0],[0],[0],[1.0],[nan],[nan],[0.0],[1.0]
4,contextOSE,train_no_anomaly,0.50,[8761],[0],[0],[0],[1.0],[nan],[nan],[0.0],[1.0]
5,contextOSE,train_with_anomaly,0.05,"[199, 492]","[492, 199]","[0, 3486]","[3486, 0]","[0.054, 1.0]","[1.0, 0.054]","[0.0, 0.946]","[0.946, 0.0]","[0.102, 0.22]"
6,contextOSE,train_with_anomaly,0.10,"[199, 492]","[492, 199]","[0, 3486]","[3486, 0]","[0.054, 1.0]","[1.0, 0.054]","[0.0, 0.946]","[0.946, 0.0]","[0.102, 0.22]"
7,contextOSE,train_with_anomaly,0.25,"[2002, 237]","[237, 2002]","[255, 1683]","[1683, 255]","[0.543, 0.482]","[0.482, 0.543]","[0.518, 0.457]","[0.457, 0.518]","[0.674, 0.197]"
8,contextOSE,train_with_anomaly,0.40,"[3685, 0]","[0, 3685]","[492, 0]","[0, 492]","[1.0, 0.0]","[0.0, 1.0]","[1.0, 0.0]","[0.0, 1.0]","[0.937, 0.0]"
9,contextOSE,train_with_anomaly,0.50,"[3685, 0]","[0, 3685]","[492, 0]","[0, 492]","[1.0, 0.0]","[0.0, 1.0]","[1.0, 0.0]","[0.0, 1.0]","[0.937, 0.0]"
